In [31]:
import dask.distributed
import pystac_client
import stackstac
import dask.array as da
from geogif import gif, dgif
import geopandas as gpd
import planetary_computer as pc

#Load STAC items into xarray Datasets. Process locally or distribute data loading and computation with Dask.
from odc.stac import configure_rio, load
from IPython.display import HTML, display
import folium
import folium.plugins
from branca.element import Figure
import shapely.geometry

In [2]:
client = dask.distributed.Client()
display(client)

/home/barreje/miniconda3/envs/geogif/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41757 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41757/status,
Dashboard: http://127.0.0.1:41757/status,Workers: 4
Total threads: 12,Total memory: 24.77 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44353,Workers: 4
Dashboard: http://127.0.0.1:41757/status,Total threads: 12
Started: Just now,Total memory: 24.77 GiB
Comm: tcp://127.0.0.1:39895,Total threads: 3
Dashboard: http://127.0.0.1:45833/status,Memory: 6.19 GiB
Nanny: tcp://127.0.0.1:34015,


In [3]:
def convert_bounds(bbox, invert_y=False):
    """
    Helper method for changing bounding box representation to leaflet notation
    ``(lon1, lat1, lon2, lat2) -> ((lat1, lon1), (lat2, lon2))``
    """
    x1, y1, x2, y2 = bbox
    if invert_y:
        y1, y2 = y2, y1
    return ((y1, x1), (y2, x2))

In [45]:
#petermann - or create one with https://geojson.io/#map=2/20.0/0.0 and follow https://aws.amazon.com/fr/blogs/apn/transforming-geospatial-data-to-cloud-native-frameworks-with-element-84-on-aws/

#filename = "geojson file path"# read in AOI as a GeoDataFrame
filename = "petermann.geojson"# read in AOI as a GeoDataFrame
# read in AOI as a GeoDataFrame
aoi = gpd.read_file(filename)

fig = Figure(width="400px", height="500px")
map1 = folium.Map()
fig.add_child(map1)

folium.GeoJson(
    aoi['geometry'],
    style_function=lambda x: dict(fill=False, weight=1, opacity=0.7, color="olive"),
    name="Query",
).add_to(map1)


map1.fit_bounds(bounds=convert_bounds(aoi.unary_union.bounds))
display(fig)

With the pystac_client module’s Client class, Open the STAC API. 

In [55]:
%%time

bbox =aoi.unary_union.bounds

LandsatSTAC = pystac_client.Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')

#for collection in LandsatSTAC.get_collections():
    #print(collection)

search = (
    LandsatSTAC
    .search(
        bbox=bbox,
        query =  {"eo:cloud_cover":{"lt":15}},
        collections = ["landsat-8-c2-l2"],
        datetime = "2020-01-01/2020-12-30"        
    )
)

CPU times: user 38.1 ms, sys: 677 µs, total: 38.8 ms
Wall time: 326 ms


In [56]:
%%time
items = pc.sign(search)
len(items)

CPU times: user 596 ms, sys: 43.6 ms, total: 640 ms
Wall time: 2.34 s


333

Turn STAC items into xarray as a temporal stack, using stackstac.


In [57]:
stack = stackstac.stack(items, bounds_latlon=bbox, epsg = 32620, resolution=30)


Then mask out bad (cloudy) pixels, according to the Sentinel-2 SCL Scene Classification Map, and take the temporal median of each quarter (three months) to hopefully get an okay-looking cloud-free frame representative of those three months.

In [58]:

#scl = stack.sel(band=["SCL"])
# Sentinel-2 Scene Classification Map: nodata, saturated/defective, dark, cloud shadow, cloud med. prob., cloud high prob., cirrus
#invalid = da.isin(scl, [0, 1, 2, 3, 8, 9, 10])
#valid = stack.where(~invalid)

rgb = stack.sel(band=["SR_B4", "SR_B3", "SR_B2"])

quarterly = rgb.resample(time="M").median()
quarterly

/home/barreje/miniconda3/envs/geogif/lib/python3.9/site-packages/dask/array/core.py:1761: PerformanceWarning: Increasing number of chunks by factor of 16
  return da_func(*args, **kwargs)


,Array,Chunk
Bytes,1.02 GiB,1.33 MiB
Shape,"(6, 3, 2696, 2826)","(1, 1, 418, 418)"
Count,33 Graph Layers,14112 Chunks
Type,float64,numpy.ndarray


In [59]:
ts = quarterly.persist()



In [ ]:
ts_local = quarterly.compute()
ts_local.plot.imshow(col="time", rgb="band", col_wrap=5, robust=True)

In [60]:

cleaned = ts.ffill("time").bfill("time")

In [61]:
gif_data = dgif(cleaned, bytes=True).compute()
with open("petermann_planetary.gif", "wb") as f:
    f.write(gif_data)


/home/barreje/miniconda3/envs/geogif/lib/python3.9/site-packages/distributed/worker.py:2938: UserWarning: Large object of size 7.33 MiB detected in task graph: 
  ([[[[["('getitem-d7d68a10fab457ab5f80c75019f65d13' ... a=np.ndarray>>)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


KeyboardInterrupt: 

2022-10-18 20:07:46,165 - distributed.worker_memory - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 5.12 GiB -- Worker memory limit: 6.19 GiB
2022-10-18 20:07:46,270 - distributed.worker_memory - WARNING - Worker is at 79% memory usage. Resuming worker. Process memory: 4.95 GiB -- Worker memory limit: 6.19 GiB
